# Project GDP
## GitHub Repo: https://github.com/JackBeerman/GDP

In [3]:
import pandas as pd
import numpy as np
import requests
import os
import zipfile
import io
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine

In [4]:
### Goal: Have a dashboard that allows us to select a country, country it trades with, view imports and exports, tarrifs
### Goal: predictive analysis of tarrifs and GDP..? too much

## Data

* The World Bank https://data.worldbank.org/
* World Integrated Trade Solution https://wits.worldbank.org/
* World Trade Organization https://www.wto.org/english/res_e/statis_e/statis_e.htm
* Our World in Data https://ourworldindata.org/trade-and-globalization
* International Monetary Fund https://data.imf.org/?sk=9d6028d4-f14a-464c-a2f2-59b2cd424b85